In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
%matplotlib inline

!pip install xgboost
import xgboost as xgb

# EDA 다시하기

In [2]:
df = pd.read_csv('train_real.csv')
df_test = pd.read_csv('test_real.csv')

print(df.shape)

(15031, 25)


In [3]:
coord = df[['lat', 'long']]
pca = PCA(n_components = 2)
pca.fit(coord)

coord_pca = pca.transform(coord)

df['coord_pca1'] = coord_pca[:, 0]
df['coord_pca2'] = coord_pca[:, 1] #원본데이터가 변형되어 새로운 feature로 만들어짐.

#test 데이터도 똑같이

coord = df_test[['lat', 'long']]
pca = PCA(n_components = 2)
pca.fit(coord)

coord_pca = pca.transform(coord)

df_test['coord_pca1'] = coord_pca[:, 0]
df_test['coord_pca2'] = coord_pca[:, 1] #원본데이터가 변형되어 새로운 feature로 만들어짐.

df_test['total_rooms'] = df_test['bathrooms'] + df_test['bedrooms']


In [10]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,per_price,total_rooms,is_renovated,living_ratio(lot),coord_pca1,coord_pca2
0,0,20141013,12.309987,3,1.00,1180,5650,1.0,0,0,...,47.5112,-122.257,1340,5650,95.857131,4.00,0,0.208850,-0.001009,-0.065768
1,1,20150225,12.100718,2,1.00,770,10000,1.0,0,0,...,47.7379,-122.233,2720,8062,63.632589,3.00,0,0.077000,-0.129219,0.122729
2,2,20150218,13.142168,3,2.00,1680,8080,1.0,0,0,...,47.6168,-122.045,1800,7503,127.832790,5.00,0,0.207921,0.092506,0.151834
3,3,20140627,12.458779,3,2.25,1715,6819,2.0,0,0,...,47.3097,-122.327,2238,6819,137.653940,5.25,0,0.251503,0.075813,-0.264767
4,4,20150115,12.583999,3,1.50,1060,9711,1.0,0,0,...,47.4095,-122.315,1650,9711,84.233957,4.50,0,0.109155,0.020466,-0.180858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15026,15030,20141014,13.322338,4,2.50,2520,6023,2.0,0,0,...,47.5137,-122.167,2520,6023,189.155985,6.50,0,0.418396,0.066050,-0.005690
15027,15031,20150326,13.822984,4,3.50,3510,7200,2.0,0,0,...,47.5537,-122.398,2050,6200,253.924920,7.50,0,0.487500,-0.136069,-0.124471
15028,15032,20140521,12.793862,3,2.50,1530,1131,3.0,0,0,...,47.6993,-122.346,1530,1509,119.588596,5.50,0,1.352785,-0.190496,0.020240
15029,15033,20150223,12.899222,4,2.50,2310,5813,2.0,0,0,...,47.5107,-122.362,1830,7200,179.080563,6.50,0,0.397385,-0.080807,-0.134014


In [11]:
def haversine_array(lat1, lng1, lat2, lng2): 
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2)) 
    AVG_EARTH_RADIUS = 6371 # in km 
    lat = lat2 - lat1 
    lng = lng2 - lng1 
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2 
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d)) 
    return h

In [12]:
print(df['lat'].min(), df['lat'].max(), df['long'].min(), df['long'].max())

haversine_dist = haversine_array(df['lat'].min(), df['long'].min(), df['lat'].max(), df['long'].max())
print(f'max distance: {haversine_dist:.2f}km')

47.1559 47.7776 -122.519 -121.315
max distance: 113.88km


In [22]:
lat2 = df['lat'].values
long2 = df['long'].values

lat1 = df.loc[0, 'lat'] # id = 0 house lat
long1 = df.loc[0, 'long'] # id = 0 house long
dist_arr = haversine_array(lat1, long1, lat2, long2)

df['neighbor_lat'] = lat2
df['neighbor_long'] = long2
df['distance'] = dist_arr


# neighbor_df = pd.DataFrame({
#     'id': np.tile(np.array([df.loc[0, 'id']]), df.shape[0]),
#     'neighbor_lat': lat2,
#     'neighbor_long': long2,
#     'distance': dist_arr,
# })

# print(neighbor_df.shape)
# neighbor_df.head()

# neighbor_df.head()

In [23]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_lot15,per_price,total_rooms,is_renovated,living_ratio(lot),coord_pca1,coord_pca2,neighbor_lat,neighbor_long,distance
0,0,20141013,12.309987,3,1.00,1180,5650,1.0,0,0,...,5650,95.857131,4.00,0,0.208850,-0.001009,-0.065768,47.5112,-122.257,0.000000
1,1,20150225,12.100718,2,1.00,770,10000,1.0,0,0,...,8062,63.632589,3.00,0,0.077000,-0.129219,0.122729,47.7379,-122.233,25.271977
2,2,20150218,13.142168,3,2.00,1680,8080,1.0,0,0,...,7503,127.832790,5.00,0,0.207921,0.092506,0.151834,47.6168,-122.045,19.771055
3,3,20140627,12.458779,3,2.25,1715,6819,2.0,0,0,...,6819,137.653940,5.25,0,0.251503,0.075813,-0.264767,47.3097,-122.327,23.016635
4,4,20150115,12.583999,3,1.50,1060,9711,1.0,0,0,...,9711,84.233957,4.50,0,0.109155,0.020466,-0.180858,47.4095,-122.315,12.120049


In [32]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_lot15,per_price,total_rooms,is_renovated,living_ratio(lot),coord_pca1,coord_pca2,neighbor_lat,neighbor_long,distance
0,0,20141013,12.309987,3,1.00,1180,5650,1.0,0,0,...,5650,95.857131,4.00,0,0.208850,-0.001009,-0.065768,47.5112,-122.257,0.000000
1,1,20150225,12.100718,2,1.00,770,10000,1.0,0,0,...,8062,63.632589,3.00,0,0.077000,-0.129219,0.122729,47.7379,-122.233,25.271977
2,2,20150218,13.142168,3,2.00,1680,8080,1.0,0,0,...,7503,127.832790,5.00,0,0.207921,0.092506,0.151834,47.6168,-122.045,19.771055
3,3,20140627,12.458779,3,2.25,1715,6819,2.0,0,0,...,6819,137.653940,5.25,0,0.251503,0.075813,-0.264767,47.3097,-122.327,23.016635
4,4,20150115,12.583999,3,1.50,1060,9711,1.0,0,0,...,9711,84.233957,4.50,0,0.109155,0.020466,-0.180858,47.4095,-122.315,12.120049


In [36]:
df.to_csv('train_real.csv', index=False)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_lot15,per_price,total_rooms,is_renovated,living_ratio(lot),coord_pca1,coord_pca2,neighbor_lat,neighbor_long,distance
0,0,20141013,12.309987,3,1.00,1180,5650,1.0,0,0,...,5650,95.857131,4.00,0,0.208850,-0.001009,-0.065768,47.5112,-122.257,0.000000
1,1,20150225,12.100718,2,1.00,770,10000,1.0,0,0,...,8062,63.632589,3.00,0,0.077000,-0.129219,0.122729,47.7379,-122.233,25.271977
2,2,20150218,13.142168,3,2.00,1680,8080,1.0,0,0,...,7503,127.832790,5.00,0,0.207921,0.092506,0.151834,47.6168,-122.045,19.771055
3,3,20140627,12.458779,3,2.25,1715,6819,2.0,0,0,...,6819,137.653940,5.25,0,0.251503,0.075813,-0.264767,47.3097,-122.327,23.016635
4,4,20150115,12.583999,3,1.50,1060,9711,1.0,0,0,...,9711,84.233957,4.50,0,0.109155,0.020466,-0.180858,47.4095,-122.315,12.120049


In [38]:
data = pd.read_csv('train_real.csv')
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_lot15,per_price,total_rooms,is_renovated,living_ratio(lot),coord_pca1,coord_pca2,neighbor_lat,neighbor_long,distance
0,0,20141013,12.309987,3,1.00,1180,5650,1.0,0,0,...,5650,95.857131,4.00,0,0.208850,-0.001009,-0.065768,47.5112,-122.257,0.000000
1,1,20150225,12.100718,2,1.00,770,10000,1.0,0,0,...,8062,63.632589,3.00,0,0.077000,-0.129219,0.122729,47.7379,-122.233,25.271977
2,2,20150218,13.142168,3,2.00,1680,8080,1.0,0,0,...,7503,127.832790,5.00,0,0.207921,0.092506,0.151834,47.6168,-122.045,19.771055
3,3,20140627,12.458779,3,2.25,1715,6819,2.0,0,0,...,6819,137.653940,5.25,0,0.251503,0.075813,-0.264767,47.3097,-122.327,23.016635
4,4,20150115,12.583999,3,1.50,1060,9711,1.0,0,0,...,9711,84.233957,4.50,0,0.109155,0.020466,-0.180858,47.4095,-122.315,12.120049


In [39]:
data['zipcode'].head()

0    98178
1    98028
2    98074
3    98003
4    98198
Name: zipcode, dtype: int64

In [41]:
#zipcode

data['zipcode'] = data['zipcode'].astype('str')
data['zipcode'].head()

0    98178
1    98028
2    98074
3    98003
4    98198
Name: zipcode, dtype: object

In [44]:
data['zipcode-3'] = data['zipcode'].str[2:3] #3번째 글자
data['zipcode-4'] = data['zipcode'].str[3:4] #4번째 글자
data['zipcode-5'] = data['zipcode'].str[4:5] #5번째 글자
data['zipcode-34'] = data['zipcode'].str[2:4] #3~4번째 글자
data['zipcode-45'] = data['zipcode'].str[3:5] #4~5번째 글자
data['zipcode-35'] = data['zipcode'].str[2:3] + data['zipcode'].str[4:5] #3번째, 5번째 글자

data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,coord_pca2,neighbor_lat,neighbor_long,distance,zipcode-3,zipcode-4,zipcode-5,zipcode-34,zipcode-45,zipcode-35
0,0,20141013,12.309987,3,1.00,1180,5650,1.0,0,0,...,-0.065768,47.5112,-122.257,0.000000,1,7,8,17,78,18
1,1,20150225,12.100718,2,1.00,770,10000,1.0,0,0,...,0.122729,47.7379,-122.233,25.271977,0,2,8,02,28,08
2,2,20150218,13.142168,3,2.00,1680,8080,1.0,0,0,...,0.151834,47.6168,-122.045,19.771055,0,7,4,07,74,04
3,3,20140627,12.458779,3,2.25,1715,6819,2.0,0,0,...,-0.264767,47.3097,-122.327,23.016635,0,0,3,00,03,03
4,4,20150115,12.583999,3,1.50,1060,9711,1.0,0,0,...,-0.180858,47.4095,-122.315,12.120049,1,9,8,19,98,18


In [45]:
data.to_csv('my_train.csv')

In [46]:
dataset = pd.read_csv('my_train.csv')
dataset.head() #변수 36개 

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,coord_pca2,neighbor_lat,neighbor_long,distance,zipcode-3,zipcode-4,zipcode-5,zipcode-34,zipcode-45,zipcode-35
0,0,0,20141013,12.309987,3,1.00,1180,5650,1.0,0,...,-0.065768,47.5112,-122.257,0.000000,1,7,8,17,78,18
1,1,1,20150225,12.100718,2,1.00,770,10000,1.0,0,...,0.122729,47.7379,-122.233,25.271977,0,2,8,2,28,8
2,2,2,20150218,13.142168,3,2.00,1680,8080,1.0,0,...,0.151834,47.6168,-122.045,19.771055,0,7,4,7,74,4
3,3,3,20140627,12.458779,3,2.25,1715,6819,2.0,0,...,-0.264767,47.3097,-122.327,23.016635,0,0,3,0,3,3
4,4,4,20150115,12.583999,3,1.50,1060,9711,1.0,0,...,-0.180858,47.4095,-122.315,12.120049,1,9,8,19,98,18


## test set revising

In [58]:
df_test.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,is_renovated,living_ratio(lot),coord_pca1,coord_pca2,total_rooms
0,15035,20141209,3,2.25,2570,7242,2.0,0,0,3,...,98125,47.7210,-122.319,1690,7639,1,0.354874,-0.188853,0.041039,5.25
1,15036,20141209,4,3.00,1960,5000,1.0,0,0,5,...,98136,47.5208,-122.393,1360,5000,0,0.392000,-0.100142,-0.153091,7.00
2,15037,20140512,4,4.50,5420,101930,1.0,0,0,3,...,98053,47.6561,-122.005,4760,101930,0,0.053174,0.079547,0.216452,8.50
3,15038,20150415,3,1.00,1780,7470,1.0,0,0,3,...,98146,47.5123,-122.337,1780,8113,0,0.238286,-0.054443,-0.119627,4.00
4,15039,20150312,3,2.50,1890,6560,2.0,0,0,3,...,98038,47.3684,-122.031,2390,7570,0,0.288110,0.263994,-0.005869,5.50


In [60]:
lat2 = df_test['lat'].values
long2 = df_test['long'].values

lat1 = df_test.loc[0, 'lat'] # id = 0 house lat
long1 = df_test.loc[0, 'long'] # id = 0 house long
dist_arr = haversine_array(lat1, long1, lat2, long2)

df_test['neighbor_lat'] = lat2
df_test['neighbor_long'] = long2
df_test['distance'] = dist_arr

df_test.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_living15,sqft_lot15,is_renovated,living_ratio(lot),coord_pca1,coord_pca2,total_rooms,neighbor_lat,neighbor_long,distance
0,15035,20141209,3,2.25,2570,7242,2.0,0,0,3,...,1690,7639,1,0.354874,-0.188853,0.041039,5.25,47.7210,-122.319,0.000000
1,15036,20141209,4,3.00,1960,5000,1.0,0,0,5,...,1360,5000,0,0.392000,-0.100142,-0.153091,7.00,47.5208,-122.393,22.941722
2,15037,20140512,4,4.50,5420,101930,1.0,0,0,3,...,4760,101930,0,0.053174,0.079547,0.216452,8.50,47.6561,-122.005,24.586451
3,15038,20150415,3,1.00,1780,7470,1.0,0,0,3,...,1780,8113,0,0.238286,-0.054443,-0.119627,4.00,47.5123,-122.337,23.245568
4,15039,20150312,3,2.50,1890,6560,2.0,0,0,3,...,2390,7570,0,0.288110,0.263994,-0.005869,5.50,47.3684,-122.031,44.771555


In [62]:
df_test['zipcode'].head()

0    98125
1    98136
2    98053
3    98146
4    98038
Name: zipcode, dtype: int64

In [63]:
df_test['zipcode'] = df_test['zipcode'].astype('str')
df_test['zipcode'].head()

0    98125
1    98136
2    98053
3    98146
4    98038
Name: zipcode, dtype: object

In [65]:
df_test['zipcode-3'] = df_test['zipcode'].str[2:3] #3번째 글자
df_test['zipcode-4'] = df_test['zipcode'].str[3:4] #4번째 글자
df_test['zipcode-5'] = df_test['zipcode'].str[4:5] #5번째 글자
df_test['zipcode-34'] = df_test['zipcode'].str[2:4] #3~4번째 글자
df_test['zipcode-45'] = df_test['zipcode'].str[3:5] #4~5번째 글자
df_test['zipcode-35'] = df_test['zipcode'].str[2:3] + df_test['zipcode'].str[4:5] #3번째, 5번째 글자

df_test.head() #34개

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,total_rooms,neighbor_lat,neighbor_long,distance,zipcode-3,zipcode-4,zipcode-5,zipcode-34,zipcode-45,zipcode-35
0,15035,20141209,3,2.25,2570,7242,2.0,0,0,3,...,5.25,47.7210,-122.319,0.000000,1,2,5,12,25,15
1,15036,20141209,4,3.00,1960,5000,1.0,0,0,5,...,7.00,47.5208,-122.393,22.941722,1,3,6,13,36,16
2,15037,20140512,4,4.50,5420,101930,1.0,0,0,3,...,8.50,47.6561,-122.005,24.586451,0,5,3,05,53,03
3,15038,20150415,3,1.00,1780,7470,1.0,0,0,3,...,4.00,47.5123,-122.337,23.245568,1,4,6,14,46,16
4,15039,20150312,3,2.50,1890,6560,2.0,0,0,3,...,5.50,47.3684,-122.031,44.771555,0,3,8,03,38,08


In [74]:
print(data.shape, df_test.shape)

(15031, 36) (6468, 34)


In [77]:
df_test.to_csv('my_test.csv')

testt = pd.read_csv('my_test.csv')
testt.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,total_rooms,neighbor_lat,neighbor_long,distance,zipcode-3,zipcode-4,zipcode-5,zipcode-34,zipcode-45,zipcode-35
0,0,15035,20141209,3,2.25,2570,7242,2.0,0,0,...,5.25,47.7210,-122.319,0.000000,1,2,5,12,25,15
1,1,15036,20141209,4,3.00,1960,5000,1.0,0,0,...,7.00,47.5208,-122.393,22.941722,1,3,6,13,36,16
2,2,15037,20140512,4,4.50,5420,101930,1.0,0,0,...,8.50,47.6561,-122.005,24.586451,0,5,3,5,53,3
3,3,15038,20150415,3,1.00,1780,7470,1.0,0,0,...,4.00,47.5123,-122.337,23.245568,1,4,6,14,46,16
4,4,15039,20150312,3,2.50,1890,6560,2.0,0,0,...,5.50,47.3684,-122.031,44.771555,0,3,8,3,38,8
